In [3]:
# pip install yfinance
# !pip install snowflake-connector-python
# !pip install snowflake-snowpark-python

In [1]:
import yfinance as yf
from tqdm import tqdm
import pandas as pd

from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import configparser
from snowflake.snowpark.session import Session

config = configparser.ConfigParser()
config.read("snowflake_config_logs.ini")

connection_params = {
    'user': f'{config["Snowflake"]["user"]}',
    'password': f'{config["Snowflake"]["password"]}',
    'account': f'{config["Snowflake"]["account"]}',
    'warehouse': f'{config["Snowflake"]["warehouse"]}',
    'database': f'{config["Snowflake"]["database"]}',
    'schema': f'{config["Snowflake"]["schema"]}',
    'role': f'{config["Snowflake"]["role"]}'
}
#print('connection_successful')
session = Session.builder.configs(connection_params).create()

stocks = {
  "Technology": [
    {"name": "Microsoft Corporation", "ticker": "MSFT"},
    {"name": "Apple Inc.", "ticker": "AAPL"},
    {"name": "Alphabet Inc.", "ticker": "GOOGL"},
    {"name": "Intel Corporation", "ticker": "INTC"},
    {"name": "Cisco Systems, Inc.", "ticker": "CSCO"}
  ],
  "Consumer Discretionary": [
    {"name": "Amazon.com, Inc.", "ticker": "AMZN"},
    {"name": "Tesla, Inc.", "ticker": "TSLA"},
    {"name": "The Home Depot, Inc.", "ticker": "HD"},
    {"name": "McDonald's Corporation", "ticker": "MCD"},
    {"name": "Nike, Inc.", "ticker": "NKE"}
  ],
  "Healthcare": [
    {"name": "Johnson & Johnson", "ticker": "JNJ"},
    {"name": "Pfizer Inc.", "ticker": "PFE"},
    {"name": "Merck & Co., Inc.", "ticker": "MRK"},
    {"name": "Abbott Laboratories", "ticker": "ABT"},
    {"name": "Amgen Inc.", "ticker": "AMGN"}
  ],
  "Financials": [
    {"name": "JPMorgan Chase & Co.", "ticker": "JPM"},
    {"name": "Bank of America Corporation", "ticker": "BAC"},
    {"name": "Wells Fargo & Company", "ticker": "WFC"},
    {"name": "Goldman Sachs Group, Inc.", "ticker": "GS"},
    {"name": "Morgan Stanley", "ticker": "MS"}
  ],
  "Industrials": [
    {"name": "Boeing Company", "ticker": "BA"},
    {"name": "Caterpillar Inc.", "ticker": "CAT"},
    {"name": "General Electric Company", "ticker": "GE"},
    {"name": "3M Company", "ticker": "MMM"},
    {"name": "Honeywell International Inc.", "ticker": "HON"}
  ],
  "Consumer Staples": [
    {"name": "The Procter & Gamble Company", "ticker": "PG"},
    {"name": "Coca-Cola Company", "ticker": "KO"},
    {"name": "PepsiCo, Inc.", "ticker": "PEP"},
    {"name": "Walmart Inc.", "ticker": "WMT"},
    {"name": "Costco Wholesale Corporation", "ticker": "COST"}
  ]
}


DatabaseError: 250001 (08001): Failed to connect to DB: fya62509.us-east-1.snowflakecomputing.com:443. Error connecting to Snowflake via Snowpark Container Services. Please use OAuth when connecting to Snowflake. For more information please refer to https://docs.snowflake.com/en/developer-guide/snowpark-container-services/additional-considerations-services-jobs#connecting-to-snowflake-from-inside-a-container.

In [37]:
stocks_list = [st["ticker"]  for key,val in stocks.items() for st in val]
counter = 1
for ticker in tqdm(stocks_list):
    if counter == 1:
        data = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        data["stock"] = [ticker]*len(data)
    else:
        print(ticker)
        df = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        df["stock"] = [ticker]*len(df)
        data = data._append(df, ignore_index=True)
        del df
    counter += 1
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AAPL
GOOGL
INTC
CSCO
AMZN
TSLA
HD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MCD
NKE
JNJ
PFE
MRK
ABT


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
 67%|██████▋   | 20/30 [00:00<00:00, 31.79it/s]

AMGN
JPM
BAC
WFC
GS
MS
BA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


CAT
GE
MMM
HON
PG
KO


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 30/30 [00:01<00:00, 28.51it/s]

PEP
WMT
COST


In [39]:
data = data[["stock","Close","Adj Close"]]

In [40]:
data

,stock,Close,Adj Close
0,MSFT,30.950001,23.347326
1,MSFT,30.959999,23.354862
2,MSFT,30.770000,23.211536
3,MSFT,30.450001,22.970146
4,MSFT,30.660000,23.128555
...,...,...,...
111233,COST,901.440002,901.440002
111234,COST,885.619995,885.619995
111235,COST,886.520020,886.520020
111236,COST,877.380005,877.380005


In [45]:
apple = yf.Ticker("AAPL")
print(apple.info)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

In [ ]:
df_snowflake=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())

df_snowflake.write.mode("overwrite").save_as_table("LOGS_ASSISTANT.SAMPLE_SCHEMA.HR_DATA")